# **Pre-training LLMs with Hugging Face**

# 
# Introduction
 
This project introduces the process of pretraining large language models (LLMs) using the popular Hugging Face library. Hugging Face is a leading open-source platform for natural language processing that provides a wide range of pretrained models and tools for fine-tuning and deploying these models.
 
This notebook covers how to load pre-trained models from Hugging Face and make inferences using the Pipeline module. Additionally, it demonstrates how to further train pre-trained LLMs on custom data through self-supervised fine-tuning. By the end of this lab, readers will have a solid understanding of how to pretrain LLMs and store them to later fine-tune for specific use cases. This will enable the creation of powerful and customized natural language processing solutions.


# __Table of Contents__

<ol>
    <li><a href="#Objectives">Objectives</a></li>
    <li>
        <a href="#Setup">Setup</a>
        <ol>
            <li><a href="#Installing-required-libraries">Installing required libraries</a></li>
            <li><a href="#Importing-required-libraries">Importing required libraries</a></li>
        </ol>
    </li>
    <li><a href="#Pretraining-and-self-supervised-fine-tuning">Pretraining and self-supervised fine-tuning</a>
        <ol>
            <li><a href="#Importing-required-datasets">Importing required datasets</a></li>
            <li><a href="#Loading-the-saved-model">Loading the saved model</a></li>
            <li><a href="#Inferencing-a-pretrained-BERT-model">Inferencing a pretrained BERT model</a></li>
        </ol>
    </li>
    <li><a href="#Exercise">Exercise</a></li>
</ol>


# Setup


### Installing required libraries
The following required libraries are pre-installed in the Skills Network Labs environment. However, if you run these notebook commands in a different Jupyter environment (e.g. Watson Studio or Ananconda), you will need to install these libraries by removing the `#` sign before `!pip` in the code cell below:

_PS: To run lab this in your own environment, please note that the versions of libraries may differ due to dependencies._


In [1]:
%pip install -U \
  torch \
  torchvision \
  transformers \
  datasets \
  accelerate \
  portalocker \
  protobuf \
  --upgrade-strategy only-if-needed \
  --prefer-binary

   ---------------------------------------- 0.0/12.0 MB ? eta -:--:--
   - -------------------------------------- 0.5/12.0 MB 5.3 MB/s eta 0:00:03
   ---------------- ----------------------- 5.0/12.0 MB 18.5 MB/s eta 0:00:01
   ---------------------------------------- 12.0/12.0 MB 26.7 MB/s  0:00:00
  Attempting uninstall: transformers
    Found existing installation: transformers 4.56.2
    Uninstalling transformers-4.56.2:
      Successfully uninstalled transformers-4.56.2
Note: you may need to restart the kernel to use updated packages.


The following required libraries are __not__ pre-installed in the Skills Network Labs environment. __You will need to run the following cell__ to install them:


In [46]:
!pip install --user datasets
# Save dataset to local path
import os
from datasets import load_dataset

# Set the local path for saving datasets
local_path = r"D:\OneDrive - Imperial College London\Langchain_LLM\Huggingface_Fine_Tuning"

# Create directory if it doesn't exist
os.makedirs(local_path, exist_ok=True)

# Load and save dataset locally, then reopen as 'dataset'
dataset = load_dataset("wikitext", "wikitext-2-raw-v1")

ERROR: Can not perform a '--user' install. User site-packages are not visible in this virtualenv.
'(ProtocolError('Connection aborted.', ConnectionResetError(10054, '远程主机强迫关闭了一个现有的连接。', None, 10054, None)), '(Request ID: ac37ece0-175d-4d08-a857-8a6c74a0e791)')' thrown while requesting HEAD https://huggingface.co/datasets/wikitext/resolve/main/README.md
Retrying in 1s [Retry 1/5].
'(ProtocolError('Connection aborted.', ConnectionResetError(10054, '远程主机强迫关闭了一个现有的连接。', None, 10054, None)), '(Request ID: d375d8fc-997d-4286-a703-36561e995125)')' thrown while requesting HEAD https://huggingface.co/datasets/Salesforce/wikitext/resolve/main/README.md
Retrying in 1s [Retry 1/5].
'(ProtocolError('Connection aborted.', ConnectionResetError(10054, '远程主机强迫关闭了一个现有的连接。', None, 10054, None)), '(Request ID: 211f9416-2861-468d-9a91-80cd47e058b2)')' thrown while requesting HEAD https://huggingface.co/datasets/Salesforce/wikitext/resolve/main/README.md
Retrying in 2s [Retry 2/5].
'(ProtocolError('Connecti

KeyboardInterrupt: 

### Importing required libraries

_It is recommended that you import all required libraries in one place (here):_
* **Note: if you got an error after running the cell below, try restarting the Kernel as some packages need a restart to be effective.**


In [1]:
import torch
from torch.optim.lr_scheduler import LambdaLR
from torch.utils.data import DataLoader
from torch.optim import AdamW
from transformers import AutoConfig,AutoModelForCausalLM,AutoModelForSequenceClassification,BertConfig,BertForMaskedLM,TrainingArguments, Trainer, TrainingArguments
from transformers import AutoTokenizer,BertTokenizerFast,TextDataset,DataCollatorForLanguageModeling
from transformers import pipeline
from datasets import load_dataset

from tqdm.auto import tqdm
import math
import time
import os


# You can also use this section to suppress warnings generated by your code:
def warn(*args, **kwargs):
    pass
import warnings
warnings.warn = warn
warnings.filterwarnings('ignore')

d:\OneDrive - Imperial College London\Langchain_LLM\Huggingface_Fine_Tuning\.venv\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


Disable tokenizer parallelism to avoid deadlocks.


In [2]:
# Set the environment variable TOKENIZERS_PARALLELISM to 'false'
import os
os.environ['TOKENIZERS_PARALLELISM'] = 'false'


---


# Pretraining and self-supervised (unsupervised) fine-tuning


Pretraining is a technique used in natural language processing (NLP) to train large language models (LLMs) on a vast corpus of unlabeled text data. The goal is to capture the general patterns and semantic relationships present in natural language, allowing the model to develop a deep understanding of language structure and meaning.

The motivation behind pretraining transformers is to address the limitations of traditional NLP approaches that often require significant amounts of labeled data for each specific task. By leveraging the abundance of unlabeled text data, pretraining enables the model to learn fundamental language skills through self-supervised objectives, facilitating transfer learning.

The pretraining objectives, such as masked language modeling (MLM) and next sentence prediction (NSP), play a crucial role in the success of transformer models. Pretrained models can be further tuned by training them on domain-specific unlabeled data, which is known as self-supervised fine-tuning.

Also, the model can be fine-tuned on specific downstream tasks using labeled data, a process known as supervised fine-tuning, further improving its performance.

In the following sections of this lab, you will explore pretraining objectives, loading pretrained models, data preparation, and the fine-tuning process. By the end, you will have a solid understanding of pretraining and self-supervised fine-tuning, empowering you to apply these techniques to solve real-world NLP problems.


In [3]:
# Load the pretrained model and tokenizer from Hugging Face
# The 'model' is the actual neural network that generates text based on input
# The 'tokenizer' converts text to tokens (numbers) that the model can understand
model = AutoModelForCausalLM.from_pretrained('facebook/opt-350m')
tokenizer = AutoTokenizer.from_pretrained('facebook/opt-350m')

# Create a text generation pipeline that combines both model and tokenizer
# The pipeline handles the complete workflow: tokenization -> model inference -> decoding
pipe = pipeline('text-generation', model = model, tokenizer=tokenizer)
print(pipe('This movie was really')[0]['generated_text'])

Xet Storage is enabled for this repo, but the 'hf_xet' package is not installed. Falling back to regular HTTP download. For better performance, install the package with: `pip install huggingface_hub[hf_xet]` or `pip install hf_xet`
Xet Storage is enabled for this repo, but the 'hf_xet' package is not installed. Falling back to regular HTTP download. For better performance, install the package with: `pip install huggingface_hub[hf_xet]` or `pip install hf_xet`
Device set to use cuda:0
Both `max_new_tokens` (=256) and `max_length`(=21) seem to have been set. `max_new_tokens` will take precedence. Please refer to the documentation for more information. (https://huggingface.co/docs/transformers/main/en/main_classes/text_generation)


This movie was really good, but I thought the first 2 seasons were pretty dull. I don't know.
It's so weird how they're both so good but the first season sucks.   It's like they didn't even watch the first season.


## Pre-training Objectives

Pre-training objectives are crucial components of the pre-training process for transformers. These objectives define the tasks that the model is trained on during the pre-training phase, allowing it to learn meaningful contextual representations of language. Three commonly used pre-training objectives are masked language modeling (MLM), next sentence prediction (NSP) and next Ttoken prediction.

1. Masked Language Modeling (MLM):
   Masked language modeling involves randomly masking some words in a sentence and training the model to predict the masked words based on the context provided by the surrounding words(i.e., words that appear either before or after the masked word). The objective is to enable the model to learn contextual understanding and fill in missing information.

2. Next Sentence Prediction (NSP):
   Next sentence prediction involves training the model to predict whether two sentences are consecutive in the original text or randomly chosen from the corpus. This objective helps the model learn sentence-level relationships and understand the coherence between sentences.

3. Next Token Prediction:
    In this objective, the model is trained to predict the next token in a sequence of text. The model is presented with a sequence of text and must learn to predict the most likely next token based on the context.

It's important to note that different pre-trained models may use variations or combinations of these objectives, depending on the specific architecture and training setup.


## Self-supervised training of a BERT model
Training a BERT(Bidirectional Encoder Representations from Transformers) model is a complex and time-consuming process that requires a large corpus of unlabeled text data and significant computational resources. However, we provide you with a simplified exercise to demonstrate the steps involved in pre-training a BERT model using the Masked Language Modeling (MLM) objective.

For this exercise, we'll use the Hugging Face Transformers library, which provides pre-implemented BERT models and tools for pre-training. You will be instructed to:
- Prepare the train dataset
- Train a Tokenizer
- Preprocess the dataset
- Pre-train BERT using an MLM task
- Evaluate the trained model


### Importing required datasets

The WikiText dataset is a widely used benchmark dataset in the field of natural language processing (NLP). The dataset contains a large amount of text extracted from Wikipedia, which is a vast online encyclopedia covering a wide range of topics. The articles in the WikiText dataset are preprocessed to remove formatting, hyperlinks, and other metadata, resulting in a clean text corpus.

The WikiText dataset has 4 different configs, and is divided into three parts: a training set, a validation set, and a test set. The training set is used for training language models, while the validation and test sets are used for evaluating the performance of the models.
First, let's load the datasets and concatenate them together to create a big dataset.

*Note: The original BERT was pretrained on Wikipedia and BookCorpus datasets.


Below files are next used in creating TextDataset objects for the training:


In [7]:
# Path to save the datasets to text files
output_file_train = "wikitext_dataset_train.txt"
output_file_test = "wikitext_dataset_test.txt"

# Open the output file in write mode
with open(output_file_train, "w", encoding="utf-8") as f:
    # Iterate over each example in the dataset
    for example in dataset["train"]:
        # Write the example text to the file
        f.write(example["text"] + "\n")

# Open the output file in write mode
with open(output_file_test, "w", encoding="utf-8") as f:
    # Iterate over each example in the dataset
    for example in dataset["test"]:
        # Write the example text to the file
        f.write(example["text"] + "\n")

You need to define a tokenizer to be used for tokenizing the dataset.


In [8]:
# create a tokenizer from existing one to re-use special tokens
bert_tokenizer = BertTokenizerFast.from_pretrained("bert-base-uncased")

In [9]:
model_name = 'bert-base-uncased'

model = AutoModelForCausalLM.from_pretrained(model_name)
tokenizer = AutoTokenizer.from_pretrained(model_name, is_decoder=True)


If you want to use `BertLMHeadModel` as a standalone, add `is_decoder=True.`


### Training a Tokenizer(Optional)

In the previous cell, you created an instance of tokenizer from a pre-trained BERT tokenizer. If you want to train the tokenizer on your own dataset, you can uncomment the code below. This is specially helpful when using transformers for specific areas such as medicine where tokens are somehow different than the general tokens that tokenizers are created based on. (You can skip this step if you do not want to train the tokenizer on your specific data):


In [10]:
## create a python generator to dynamically load the data
def batch_iterator(batch_size=10000):
    # Create a generator function that yields batches of text data from the training dataset
    # batch_size: number of examples to include in each batch (default 10000)
    
    for i in tqdm(range(0, len(dataset), batch_size)):
        # Iterate through the dataset in chunks of batch_size
        # tqdm provides a progress bar to track iteration progress
        # range(0, len(dataset), batch_size) creates indices: 0, 10000, 20000, etc.
        
        yield dataset['train'][i : i + batch_size]["text"]
        # Yield a batch of text examples from the training split
        # Slices the dataset from index i to i+batch_size
        # Extracts only the "text" field from each example in the batch
        # yield makes this a generator, returning one batch at a time on demand

## create a tokenizer from existing one to re-use special tokens
bert_tokenizer = BertTokenizerFast.from_pretrained("bert-base-uncased")

## train the tokenizer using our own dataset
bert_tokenizer = bert_tokenizer.train_new_from_iterator(text_iterator=batch_iterator(), vocab_size=30522)

'(ProtocolError('Connection aborted.', ConnectionResetError(10054, '远程主机强迫关闭了一个现有的连接。', None, 10054, None)), '(Request ID: 0565c648-619c-4ec5-a7bf-81acf7e56d66)')' thrown while requesting HEAD https://huggingface.co/bert-base-uncased/resolve/main/tokenizer_config.json
Retrying in 1s [Retry 1/5].
'(ProtocolError('Connection aborted.', ConnectionResetError(10054, '远程主机强迫关闭了一个现有的连接。', None, 10054, None)), '(Request ID: a587dee8-12cf-4298-87b7-a8add89fc98d)')' thrown while requesting HEAD https://huggingface.co/bert-base-uncased/resolve/main/tokenizer_config.json
Retrying in 2s [Retry 2/5].
'(ProtocolError('Connection aborted.', ConnectionResetError(10054, '远程主机强迫关闭了一个现有的连接。', None, 10054, None)), '(Request ID: 36b5f084-d7b1-4f14-a83f-7ca1be61b8c3)')' thrown while requesting HEAD https://huggingface.co/bert-base-uncased/resolve/main/tokenizer_config.json
Retrying in 4s [Retry 3/5].
100%|██████████| 1/1 [00:00<00:00,  1.06it/s]


### Pretraining

In this step, we define the configuration of the BERT model and create the model:
#### Define the BERT Configuration
Here, we define the configuration settings for a BERT model using `BertConfig`. This includes setting various parameters related to the model's architecture:
- **vocab_size=30522**: Specifies the size of the vocabulary. This number should match the vocabulary size used by the tokenizer.
- **hidden_size=768**: Sets the size of the hidden layers.
- **num_hidden_layers=12**: Determines the number of hidden layers in the transformer model.
- **num_attention_heads=12**: Sets the number of attention heads in each attention layer.
- **intermediate_size=3072**: Specifies the size of the "intermediate" (i.e., feed-forward) layer within the transformer.



In [13]:
# Define the BERT configuration
config = BertConfig(
    vocab_size=len(bert_tokenizer.get_vocab()),  # Specify the vocabulary size(Make sure this number equals the vocab_size of the tokenizer)
    hidden_size=768,  # Set the hidden size
    num_hidden_layers=12,  # Set the number of layers
    num_attention_heads=12,  # Set the number of attention heads
    intermediate_size=3072,  # Set the intermediate size
)

 Create the BERT model for pre-training:


In [14]:
# Create the BERT model for pre-training
model = BertForMaskedLM(config)

check model configuration


In [15]:
# check model configuration
model

BertForMaskedLM(
  (bert): BertModel(
    (embeddings): BertEmbeddings(
      (word_embeddings): Embedding(30522, 768, padding_idx=0)
      (position_embeddings): Embedding(512, 768)
      (token_type_embeddings): Embedding(2, 768)
      (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (encoder): BertEncoder(
      (layer): ModuleList(
        (0-11): 12 x BertLayer(
          (attention): BertAttention(
            (self): BertSdpaSelfAttention(
              (query): Linear(in_features=768, out_features=768, bias=True)
              (key): Linear(in_features=768, out_features=768, bias=True)
              (value): Linear(in_features=768, out_features=768, bias=True)
              (dropout): Dropout(p=0.1, inplace=False)
            )
            (output): BertSelfOutput(
              (dense): Linear(in_features=768, out_features=768, bias=True)
              (LayerNorm): LayerNorm((768,), eps=1e-12, elementwi

### Tokenize Dataset Dynamically

In this section, we dynamically tokenize the dataset 
This approach provides greater flexibility and integrates well with modern NLP workflows.

#### **Tokenization Function**
The `tokenize_function` is used to preprocess the text data by tokenizing and formatting it for model training.


In [19]:
# Tokenize dataset dynamically
def tokenize_function(examples):
    return bert_tokenizer(examples["text"], truncation=True, padding="max_length", max_length=512)

# Tokenize train and test datasets
tokenized_datasets = dataset.map(tokenize_function, batched=True, remove_columns=["text"])

# Print tokenized dataset sample
print(tokenized_datasets["train"][0])

# Split into training and test sets
train_dataset = tokenized_datasets["train"]
test_dataset = tokenized_datasets["test"]

Map: 100%|██████████| 4358/4358 [00:01<00:00, 3067.51 examples/s]

{'input_ids': [2, 3, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0

Then, we prepare data for the MLM task (masking random tokens):
### Define the Data Collator for MLM task
This line of code sets up a `DataCollatorForLanguageModeling` from the Hugging Face Transformers library. A data collator is used during training to dynamically create batches of data. For language modeling, particularly for models like BERT that use masked language modeling (MLM), this collator prepares training batches by automatically masking tokens according to a specified probability. Here are the details of the parameters used:

- **tokenizer=bert_tokenizer**: Specifies the tokenizer to be used with the data collator. The `bert_tokenizer` is responsible for tokenizing the text and converting it to the format expected by the model.
- **mlm=True**: Indicates that the data collator should mask tokens for masked language modeling training. This parameter being set to `True` configures the collator to randomly mask some of the tokens in the input data, which the model will then attempt to predict.
- **mlm_probability=0.15**: Sets the probability with which tokens will be masked. A probability of 0.15 means that, on average, 15% of the tokens in any sequence will be replaced with a mask token.


In [22]:
# Prepare the data collator for language modeling
data_collator = DataCollatorForLanguageModeling(
    tokenizer=bert_tokenizer, mlm=True, mlm_probability=0.15
)

### The masked tokenized training data

In [23]:
# check how collator transforms a sample input data record
data_collator([train_dataset[0]])

{'input_ids': tensor([[2, 3, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
         0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
         0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
         0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
         0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
         0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
         0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
         0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
         0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
         0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
         0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
         0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
         0, 0,

Now, we train the BERT Model using the Trainer module. (For a complete list of training arguments, check [here](https://huggingface.co/docs/transformers/v4.33.2/en/main_classes/trainer#transformers.TrainingArguments)):
This section configures the training process by specifying various parameters that control how the model is trained, evaluated, and saved:

- **output_dir="./trained_model"**: Specifies the directory where the trained model and other output files will be saved.
- **overwrite_output_dir=True**: If set to `True`, this will overwrite the contents of the output directory if it already exists. This is useful when running experiments multiple times.
- **do_eval=True**: Enables evaluation of the model. If `True`, the model will be evaluated at the specified intervals.
- **evaluation_strategy="epoch"**: Defines when the model should be evaluated. Setting this to "epoch" means the model will be evaluated at the end of each epoch.
- **learning_rate=5e-5**: Sets the learning rate for training the model. This is a typical learning rate for fine-tuning BERT-like models.
- **num_train_epochs=10**: Specifies the number of training epochs. Each epoch involves a full pass over the training data.
- **per_device_train_batch_size=2**: Sets the batch size for training on each device. This should be set based on the memory capacity of your hardware.
- **save_total_limit=2**: Limits the total number of model checkpoints to be saved. Only the most recent two checkpoints will be kept.
- **logging_steps=20**: Determines how often to log training information, which can help monitor the training process.


In [34]:
# Define the training arguments
training_args = TrainingArguments(
    output_dir="./trained_model",  # Specify the output directory for the trained model
    overwrite_output_dir=True,
    do_eval=True,
    eval_strategy="epoch",  # Changed from evaluation_strategy to eval_strategy
    learning_rate=5e-7,
    num_train_epochs=10,  # Specify the number of training epochs
    per_device_train_batch_size=2,  # Set the batch size for training
    save_total_limit=2,  # Limit the total number of saved checkpoints
    logging_steps = 20
    
)

# Limit the training dataset to 500 samples
train_dataset_subset = train_dataset.select(range(100))
test_dataset_subset = test_dataset.select(range(50))

# Instantiate the Trainer
trainer = Trainer(
    model=model,
    args=training_args,
    data_collator=data_collator,
    train_dataset=train_dataset_subset,
    eval_dataset=test_dataset_subset,
)

# Start the pre-training
trainer.train()

Epoch,Training Loss,Validation Loss
1,5.691000,7.174203
2,5.234300,nan
3,5.349200,6.749716
4,6.164900,6.709776
5,5.489800,7.690262
6,5.690800,7.212485
7,5.617000,nan
8,5.968600,nan


KeyboardInterrupt: 

## Analysis of Training and Validation Results

As a professional data scientist, let me analyze the training and validation results from the 10 epochs of BERT pre-training:

### Training Loss Analysis:
- **Initial Loss**: Started at 5.487 in epoch 1
- **Final Loss**: Ended at 5.055 in epoch 10
- **Pattern**: The training loss shows high volatility with fluctuations between ~5.0-6.7
- **Overall Trend**: Minimal improvement over 10 epochs (only ~0.4 reduction)

### Validation Loss Analysis:
- **Range**: Varies between 6.5-7.9 when not NaN
- **NaN Values**: Epochs 2 and 7 show NaN validation losses, indicating computational issues
- **Pattern**: No clear decreasing trend; validation loss remains consistently higher than training loss

### Key Issues and Explanations:

#### 1. **Insufficient Training Data**
- Only 100 training samples and 50 validation samples were used
- BERT requires massive datasets (millions of sentences) for effective pre-training
- Small dataset leads to overfitting and poor generalization

#### 2. **Learning Rate Issues**
- Learning rate of 5e-5 might be too high for stable convergence
- High learning rate can cause loss oscillations and prevent convergence

#### 3. **Batch Size Limitations**
- Batch size of 2 is extremely small for BERT training
- Small batches lead to noisy gradient estimates and unstable training

#### 4. **Model Complexity vs. Data Size**
- BERT-base has 110M parameters but trained on only 100 samples
- Severe overfitting occurs when model capacity far exceeds data size

#### 5. **NaN Validation Losses**
- Indicates numerical instability during training
- Could be caused by gradient explosion or division by zero
- Suggests need for gradient clipping or learning rate adjustment

### Recommendations for Improvement:
1. **Increase Dataset Size**: Use at least 10K-100K samples for meaningful pre-training
2. **Reduce Learning Rate**: Try 1e-5 or 2e-5 for more stable training
3. **Increase Batch Size**: Use batch size of 16-32 if hardware permits
4. **Add Gradient Clipping**: Prevent gradient explosion
5. **Use Learning Rate Scheduling**: Implement warmup and decay strategies
6. **Monitor Training Stability**: Add early stopping and better logging

The current results demonstrate why BERT pre-training typically requires substantial computational resources and large-scale datasets to achieve meaningful language understanding.


## Evaluating Model Performance

Let's check the performance of the trained model. Perplexity is commonly used to compare different language models or different configurations of the same model.
After training, perplexity can be calculated on a held-out evaluation dataset to assess the model's performance. The perplexity is calculated by feeding the evaluation dataset through the model and comparing the predicted probabilities of the target tokens with the actual token values that are masked.

A lower perplexity score indicates that the model has a better understanding of the language and is more effective at predicting the masked tokens. It suggests that the model has learned useful representations and can generalize well to unseen data.


In [35]:
eval_results = trainer.evaluate()
print(f"Perplexity:{math.exp(eval_results['eval_loss']):.2f}")

Perplexity:1005.01


## Loading the saved model
If you want to skip training and load the model that you trained for 10 epochs, go ahead and uncomment the following cell:


In [ ]:
#!wget 'https://cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud/BeXRxFT2EyQAmBHvxVaMYQ/bert-scratch-model.pt'
#model.resize_token_embeddings(30522)
#model.load_state_dict(torch.load('bert-scratch-model.pt',map_location=torch.device('cpu')))

The simplest way to try out the model for inference is to use it in a pipeline(). Instantiate a pipeline for fill-mask with your model, and pass your text to it. If you like, you can use the top_k parameter to specify how many predictions to return:


In [37]:
# Define the input text with a masked token
text = "This is a [MASK] movie!"

# Create a pipeline for the "fill-mask" task
mask_filler = pipeline("fill-mask", model=model,tokenizer=bert_tokenizer)

# Generate predictions by filling the mask in the input text
results = mask_filler(text) #top_k parameter can be set 

# Print the predicted sequences
for result in results:
    print(f"Predicted token: {result['token_str']}, Confidence: {result['score']:.2f}")

Device set to use cuda:0


Predicted token: the, Confidence: 0.06
Predicted token: ,, Confidence: 0.04
Predicted token: ., Confidence: 0.03
Predicted token: of, Confidence: 0.02
Predicted token: and, Confidence: 0.02


You can see that [MASK] is replaced by the most frequent token. This weak performance can be due to insufficient training, lack of training data, model architecture, or not tuning hyperparameters. Let's try a pretrained model from Hugging Face:


## Inferencing a pretrained BERT model


In [38]:
# Load the pretrained BERT model and tokenizer
pretrained_model = BertForMaskedLM.from_pretrained('bert-base-uncased')
pretrained_tokenizer = BertTokenizerFast.from_pretrained('bert-base-uncased')

# Define the input text with a masked token
text = "This is a [MASK] movie!"

# Create the pipeline
mask_filler = pipeline(task='fill-mask', model=pretrained_model,tokenizer=pretrained_tokenizer)

# Perform inference using the pipeline
results = mask_filler(text)
for result in results:
    print(f"Predicted token: {result['token_str']}, Confidence: {result['score']:.2f}")

'(ProtocolError('Connection aborted.', ConnectionResetError(10054, '远程主机强迫关闭了一个现有的连接。', None, 10054, None)), '(Request ID: bc96775a-67e2-4bf2-8b97-f2db12674334)')' thrown while requesting HEAD https://huggingface.co/bert-base-uncased/resolve/main/config.json
Retrying in 1s [Retry 1/5].
'(ProtocolError('Connection aborted.', ConnectionResetError(10054, '远程主机强迫关闭了一个现有的连接。', None, 10054, None)), '(Request ID: 0a3f87b5-8487-42b4-8311-ba58c42fb85e)')' thrown while requesting HEAD https://huggingface.co/bert-base-uncased/resolve/main/config.json
Retrying in 2s [Retry 2/5].
'(ProtocolError('Connection aborted.', ConnectionResetError(10054, '远程主机强迫关闭了一个现有的连接。', None, 10054, None)), '(Request ID: baec9ea4-343b-4cc8-910a-80e305ea7d98)')' thrown while requesting HEAD https://huggingface.co/bert-base-uncased/resolve/main/config.json
Retrying in 4s [Retry 3/5].
'(ProtocolError('Connection aborted.', ConnectionResetError(10054, '远程主机强迫关闭了一个现有的连接。', None, 10054, None)), '(Request ID: 2472f1ba-5257-4a

Predicted token: great, Confidence: 0.16
Predicted token: horror, Confidence: 0.08
Predicted token: good, Confidence: 0.08
Predicted token: bad, Confidence: 0.05
Predicted token: fantastic, Confidence: 0.04


This pretrianed model performs way better than the model you just trained for a few epochs using a single dataset. Still, pretrained models cannot be used for specific tasks, such as sentiment extraction or sequence classification. This is why supervised fine-tuning methods are introduced.


## Exercise


1. Create a model and tokenizer using Hugging Face library.
2. Go to this [link](https://huggingface.co/datasets?task_categories=task_categories:text-classification&sort=trending)
3. Choose a Text Classification dataset that you can load, for instance 'stanfordnlp/snli'
4. Use that dataset to train your model(please be mindful of the resources available for the training) and evaluate it.

   >Note: The lab environment doesn't have the resources to support the training and this might cause the kernel to die.


In [45]:
cla_dataset = load_dataset("stanfordnlp/snli")
cla_dataset

ConnectionError: (ProtocolError('Connection aborted.', ConnectionResetError(10054, '远程主机强迫关闭了一个现有的连接。', None, 10054, None)), '(Request ID: 25c9b554-464d-4561-b2d9-902815a7ad4a)')

<details><summary>Click here for a hint</summary>

-   SNLI has 3 labels
-   You can use `load_dataset("stanfordnlp/snli")` to load the dataset
</details>


<details><summary>Click here for the solution</summary>

```python
from transformers import AutoTokenizer, AutoModelForSequenceClassification
from datasets import load_dataset

# Load the SNLI dataset
snli = load_dataset("stanfordnlp/snli")

# Preprocessing function
def preprocess_function(examples):
  premise = examples["premise"]
  hypothesis = examples["hypothesis"]
  return tokenizer(premise, hypothesis, padding="max_length", truncation=True)

model_name = "bert-base-uncased"

# Load tokenizer and model
tokenizer = AutoTokenizer.from_pretrained(model_name)
model = AutoModelForSequenceClassification.from_pretrained(model_name, num_labels=3)

# Apply preprocessing to training and validation sets
train_encoded = snli["train"].map(preprocess_function, batched=True)
val_encoded = snli["validation"].map(preprocess_function, batched=True)

# Training function (replace with your training loop)
from transformers import TrainingArguments, Trainer

training_args = TrainingArguments(
    output_dir="./results",  # Replace with your output directory
    per_device_train_batch_size=16,
    per_device_eval_batch_size=16,
    num_train_epochs=3,
)

trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=train_encoded,
    eval_dataset=val_encoded,
)

trainer.train()

# Evaluation function (replace with your metrics)
from sklearn.metrics import accuracy_score

predictions, labels = trainer.predict(val_encoded)
accuracy = accuracy_score(labels, predictions.argmax(-1))
print(f"Accuracy on validation set: {accuracy:.4f}")

```

</details>


In [ ]:
from transformers import AutoTokenizer, AutoModelForSequenceClassification
from datasets import load_dataset

# Load the SNLI dataset
snli = load_dataset("stanfordnlp/snli")

# Preprocessing function
def preprocess_function(examples):
  premise = examples["premise"]
  hypothesis = examples["hypothesis"]
  return tokenizer(premise, hypothesis, padding="max_length", truncation=True)

model_name = "bert-base-uncased"

# Load tokenizer and model
tokenizer = AutoTokenizer.from_pretrained(model_name)
model = AutoModelForSequenceClassification.from_pretrained(model_name, num_labels=3)

# Apply preprocessing to training and validation sets
train_encoded = snli["train"].map(preprocess_function, batched=True)
val_encoded = snli["validation"].map(preprocess_function, batched=True)

# Training function (replace with your training loop)
from transformers import TrainingArguments, Trainer

training_args = TrainingArguments(
    output_dir="./results",  # Replace with your output directory
    per_device_train_batch_size=2,
    per_device_eval_batch_size=2,
    num_train_epochs=1,
)

trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=train_encoded,
    eval_dataset=val_encoded,
)

trainer.train()

# Evaluation function (replace with your metrics)
from sklearn.metrics import accuracy_score

predictions, labels = trainer.predict(val_encoded)
accuracy = accuracy_score(labels, predictions.argmax(-1))
print(f"Accuracy on validation set: {accuracy:.4f}")